# installation dependances

In [ ]:
import os

print(os.environ.keys())

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

# configuration du model

In [ ]:
from unsloth import FastModel
import torch
max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-pt-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)




In [ ]:

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = False,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [ ]:
from datasets import load_dataset

# Load CSV into Hugging Face dataset
dataset = load_dataset("csv", data_files="fr-train-dataset2.csv")

# Access the split (train by default)
train_dataset = dataset["train"]

# Preview
print(train_dataset[0])
train_dataset

In [ ]:
train_dataset[0]["Address"]

In [ ]:
# Define columns to exclude from tgt
exclude_cols = ["Address", "Street_Name_old"]  # 👈 Add any column to ignore here

# Determine tgt columns
tgt_cols = [col for col in train_dataset.column_names if col not in exclude_cols]
src_col = "Address"


# Transform
def transform(example):
    return {
        "question": example[src_col],
        "answer": {col: example[col] for col in tgt_cols}
    }

dataset_refined = train_dataset.map(transform)


# Preview
print(dataset_refined[0])


print(dataset_refined.column_names)


In [ ]:
dataset_refined[0]["answer"]

In [ ]:
dataset_refined[0]["question"]

# configuration de l'entrainement

In [ ]:
import json

def format_example(example):
    return {
        "text": f"Adresse : {example['question']}\nChamps : {json.dumps(example['answer'], ensure_ascii=False)}"
    }

formatted_dataset = dataset_refined.map(format_example)
formatted_dataset = formatted_dataset.remove_columns(
    [col for col in formatted_dataset.column_names if col != "text"]
)

print(formatted_dataset[0])



In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=SFTConfig(
        max_seq_length=1024,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)


In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#trainer.save_model("gemma3-address-parser")
model.save_pretrained("gemma3-address-parser") 
tokenizer.save_pretrained("gemma3-address-parser")
model.config.save_pretrained("gemma3-address-parser")

model.save_pretrained_gguf(
        "gemma3-address-parser",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

# run it

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("gemma3-address-parser", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("gemma3-address-parser", torch_dtype=torch.float16).cuda()


In [ ]:
address = "Leclerc, 10 avenue Victor Hugo, 92200 Neuilly-sur-Seine"

prompt = f"Adresse : {address}\nChamps :"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
